In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/child-mind-institute-problematic-internet-use/sample_submission.csv
/kaggle/input/child-mind-institute-problematic-internet-use/data_dictionary.csv
/kaggle/input/child-mind-institute-problematic-internet-use/train.csv
/kaggle/input/child-mind-institute-problematic-internet-use/test.csv
/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet/id=00115b9f/part-0.parquet
/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet/id=001f3379/part-0.parquet
/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=0745c390/part-0.parquet
/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=eaab7a96/part-0.parquet
/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=8ec2cc63/part-0.parquet
/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=b2987a65/part-0.parquet
/kaggle/input/child-mind-institute-problematic-intern

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.neural_network import MLPClassifier 
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB

In [3]:
train_data_path = '/kaggle/input/child-mind-institute-problematic-internet-use/train.csv'
test_data_path = '/kaggle/input/child-mind-institute-problematic-internet-use/test.csv'

train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)

data_dict = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/data_dictionary.csv')
sample_submission = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/sample_submission.csv')

train_data.dropna(axis=0, subset=['sii'], inplace=True)

In [4]:
data_dict

,Instrument,Field,Description,Type,Values,Value Labels
0,Identifier,id,Participant's ID,str,NaN,NaN
1,Demographics,Basic_Demos-Enroll_Season,Season of enrollment,str,"Spring, Summer, Fall, Winter",NaN
2,Demographics,Basic_Demos-Age,Age of participant,float,NaN,NaN
3,Demographics,Basic_Demos-Sex,Sex of participant,categorical int,"0,1","0=Male, 1=Female"
4,Children's Global Assessment Scale,CGAS-Season,Season of participation,str,"Spring, Summer, Fall, Winter",NaN
...,...,...,...,...,...,...
76,Sleep Disturbance Scale,SDS-Season,Season of participation,str,"Spring, Summer, Fall, Winter",NaN
77,Sleep Disturbance Scale,SDS-SDS_Total_Raw,Total Raw Score,int,NaN,NaN
78,Sleep Disturbance Scale,SDS-SDS_Total_T,Total T-Score,int,NaN,NaN
79,Internet Use,PreInt_EduHx-Season,Season of participation,str,"Spring, Summer, Fall, Winter",NaN


In [5]:
train_data.head()

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,4.0,2.0,4.0,55.0,NaN,NaN,NaN,Fall,3.0,2.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,...,0.0,0.0,0.0,0.0,Fall,46.0,64.0,Summer,0.0,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,2.0,1.0,1.0,28.0,Fall,38.0,54.0,Summer,2.0,0.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,3.0,4.0,1.0,44.0,Summer,31.0,45.0,Winter,0.0,1.0
5,001f3379,Spring,13,1,Winter,50.0,Summer,22.279952,59.5,112.2,...,1.0,2.0,1.0,34.0,Summer,40.0,56.0,Spring,0.0,1.0


In [6]:
test_data.head()

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,BIA-BIA_TBW,PAQ_A-Season,PAQ_A-PAQ_A_Total,PAQ_C-Season,PAQ_C-PAQ_C_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,32.6909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fall,3.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,...,27.0552,NaN,NaN,Fall,2.340,Fall,46.0,64.0,Summer,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,NaN,NaN,NaN,Summer,2.170,Fall,38.0,54.0,Summer,2.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,45.9966,NaN,NaN,Winter,2.451,Summer,31.0,45.0,Winter,0.0
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,Summer,1.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
y_train_full = train_data.sii
X_train_full = train_data.drop(columns=['sii', 'id'], axis=1)

X_test = test_data

#Find the common columns
common_columns = X_train_full.columns.intersection(X_test.columns)

#Select only the common columns in both datasets
X_train_full = X_train_full[common_columns]
X_test = X_test[common_columns]

In [8]:

# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if 
                    X_train_full[cname].dtype == "object"]

# Select numerical column
numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

my_cols = categorical_cols + numerical_cols
print(len(my_cols))
print(len(X_test.columns)) 


58
58


In [9]:
#Split train data
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

In [10]:
X_train.describe()

,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,...,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_A-PAQ_A_Total,PAQ_C-PAQ_C_Total,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday
count,2188.000000,2188.000000,1866.000000,2024.000000,2027.000000,2059.000000,392.000000,1978.000000,1986.000000,1978.000000,...,1467.000000,1467.000000,1467.000000,1467.000000,1467.000000,281.000000,1146.000000,2029.000000,2027.000000,2123.000000
mean,10.169561,0.356947,65.015005,19.102846,55.770271,87.444842,26.676020,69.637007,82.060423,117.064206,...,33.022258,20.081133,66.566808,33.994241,53.780154,2.210715,2.600400,40.920651,57.561421,1.003768
std,3.388440,0.479209,11.835141,4.933160,7.380755,43.531736,5.273069,13.884876,13.840705,17.234625,...,64.310981,81.067935,123.940819,94.989420,148.839683,0.796155,0.789695,10.375880,13.195304,1.078880
min,5.000000,0.000000,25.000000,0.000000,36.000000,0.000000,19.000000,11.000000,33.000000,49.000000,...,17.845000,5.470790,23.620100,11.382500,21.617300,0.990000,0.580000,17.000000,38.000000,0.000000
25%,8.000000,0.000000,59.000000,15.727415,50.000000,57.000000,23.000000,61.000000,73.000000,107.000000,...,24.274950,12.824050,44.724100,20.985250,35.574600,1.550000,2.012500,33.000000,47.000000,0.000000
50%,9.000000,0.000000,65.000000,17.734372,55.000000,75.600000,26.000000,68.000000,81.000000,114.000000,...,28.597600,16.151700,55.871000,26.869400,44.525300,2.090000,2.560000,39.000000,55.000000,1.000000
75%,12.000000,1.000000,75.000000,21.151773,61.500000,110.300000,29.000000,75.000000,91.000000,125.000000,...,34.920000,21.670350,75.121150,37.058200,58.761600,2.780000,3.160000,46.000000,64.000000,2.000000
max,22.000000,1.000000,95.000000,46.102914,78.500000,315.000000,50.000000,179.000000,138.000000,203.000000,...,2457.910000,3108.170000,4683.710000,3607.690000,5690.910000,4.520000,4.790000,96.000000,100.000000,3.000000


In [11]:
for col in categorical_cols:
    print(col, ' : ', len(X_train[col].unique()))

Basic_Demos-Enroll_Season  :  4
CGAS-Season  :  5
Physical-Season  :  5
Fitness_Endurance-Season  :  5
FGC-Season  :  5
BIA-Season  :  5
PAQ_A-Season  :  5
PAQ_C-Season  :  5
SDS-Season  :  5
PreInt_EduHx-Season  :  5


In [12]:
# Preprocessing for numerical data: Impute missing values and scale features
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))
])

# Preprocessing for categorical data: One-hot encode
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# Define model with a neural network
# Define model with a neural network
model = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=1000, random_state=42, alpha = 0.01, activation = 'relu', learning_rate_init = 0.01, solver = 'adam')

# Create the pipeline
clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

'''# Định nghĩa tham số grid cho GridSearchCV
param_grid = {
    'model__hidden_layer_sizes': [(64, 32), (128, 64), (64, 64), (32, 32)],  # Các cấu trúc lớp ẩn khác nhau
    'model__activation': ['relu', 'tanh'],  # Chọn activation function
    'model__solver': ['adam', 'sgd'],  # Các thuật toán tối ưu khác nhau
    'model__learning_rate_init': [0.001, 0.01, 0.1],  # Hệ số học
    'model__alpha': [0.0001, 0.001, 0.01]  # Regularization (L2 penalty)
}

# Tạo GridSearchCV
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)

# Huấn luyện với GridSearchCV
grid_search.fit(X_train_full, y_train_full)

# In kết quả tốt nhất
print("Best parameters found: ", grid_search.best_params_)

# Đánh giá mô hình với dữ liệu validation
best_model = grid_search.best_estimator_'''


'''Fitting 5 folds for each of 144 candidates, totalling 720 fits'
'Best parameters found:  '{'model__activation': 'relu', 'model__alpha': 0.01, 'model__hidden_layer_sizes': (64, 32), 'model__learning_rate_init': 0.01, 'model__solver': 'adam'}
'''

"Fitting 5 folds for each of 144 candidates, totalling 720 fits'\n'Best parameters found:  '{'model__activation': 'relu', 'model__alpha': 0.01, 'model__hidden_layer_sizes': (64, 32), 'model__learning_rate_init': 0.01, 'model__solver': 'adam'}\n"

In [13]:
clf.fit(X_train_full, y_train_full)
pre = clf.predict(X_valid)
print(mean_absolute_error(y_valid, pre))

0.5291970802919708


In [14]:
predicts = clf.predict(X_test)
print(predicts)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [15]:
sample_submission

,id,sii
0,00008ff9,0
1,000fd460,1
2,00105258,2
3,00115b9f,3
4,0016bb22,0
5,001f3379,1
6,0038ba98,2
7,0068a485,3
8,0069fbed,0
9,0083e397,1


In [16]:
output = pd.DataFrame({
    'id': test_data['id'],  # Use 'id' from the test_data (not index)
    'sii': predicts  # The predictions you obtained for the 'sii' values
})

# Save the predictions to CSV in the correct format
output.to_csv('/kaggle/working/submission.csv', index=False)

# Check the first few rows to verify it's correct
print(output)

          id  sii
0   00008ff9  0.0
1   000fd460  0.0
2   00105258  0.0
3   00115b9f  0.0
4   0016bb22  0.0
5   001f3379  0.0
6   0038ba98  0.0
7   0068a485  0.0
8   0069fbed  0.0
9   0083e397  0.0
10  0087dd65  0.0
11  00abe655  0.0
12  00ae59c9  0.0
13  00af6387  0.0
14  00bd4359  0.0
15  00c0cd71  0.0
16  00d56d4b  0.0
17  00d9913d  0.0
18  00e6167c  0.0
19  00ebc35d  0.0
